In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define recall metric
def recall_m(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

# Define precision metric
def precision_m(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

# Define F1 score metric
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [13]:
train_data = train_datagen.flow_from_directory(
    'C:/Users/kvsud/OneDrive/Desktop/ML/DL/Data/train',  # Parent directory containing Landslide and Non-Landslides folders
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 1272 images belonging to 2 classes.


In [14]:
val_data = train_datagen.flow_from_directory(
    'C:/Users/kvsud/OneDrive/Desktop/ML/DL/Data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 318 images belonging to 2 classes.


In [8]:
# Define the U-Net model for binary classification
def unet_classification_model():
    inputs = tf.keras.layers.Input((224, 224, 3))

    # Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    # Flatten and add dense layers for binary classification
    flattened = tf.keras.layers.Flatten()(c9)
    dense1 = tf.keras.layers.Dense(64, activation='relu')(flattened)
    dropout = tf.keras.layers.Dropout(0.5)(dense1)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)  # Sigmoid for binary classification

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

    return model


In [16]:
model = unet_classification_model()

In [17]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20  # Adjust the number of epochs as needed
)

Epoch 1/20
40/40 [==============================] - 416s 10s/step - loss: 2.2986 - accuracy: 0.5134 - f1_m: 0.5172 - precision_m: 0.5343 - recall_m: 0.5595 - val_loss: 0.6927 - val_accuracy: 0.5912 - val_f1_m: 0.5763 - val_precision_m: 0.6213 - val_recall_m: 0.5444
Epoch 2/20
40/40 [==============================] - 290s 7s/step - loss: 0.6851 - accuracy: 0.5582 - f1_m: 0.6064 - precision_m: 0.5550 - recall_m: 0.6927 - val_loss: 0.6884 - val_accuracy: 0.5063 - val_f1_m: 0.1833 - val_precision_m: 0.6717 - val_recall_m: 0.1089
Epoch 3/20
40/40 [==============================] - 294s 7s/step - loss: 0.6687 - accuracy: 0.5928 - f1_m: 0.6119 - precision_m: 0.6051 - recall_m: 0.6654 - val_loss: 0.6497 - val_accuracy: 0.7075 - val_f1_m: 0.6432 - val_precision_m: 0.8315 - val_recall_m: 0.5348
Epoch 4/20
40/40 [==============================] - 381s 9s/step - loss: 0.6246 - accuracy: 0.6541 - f1_m: 0.6831 - precision_m: 0.6368 - recall_m: 0.7661 - val_loss: 0.5656 - val_accuracy: 0.8082 - val_f

In [18]:
model.evaluate(val_data)

10/10 [==============================] - 7s 651ms/step - loss: 0.6128 - accuracy: 0.7925 - f1_m: 0.7755 - precision_m: 0.8241 - recall_m: 0.7376


[0.6128371357917786,
 0.7924528121948242,
 0.775530219078064,
 0.8240969777107239,
 0.7376168370246887]